In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import api_keys

# Import API key
from api_keys import g_key

In [2]:
weather_csv = "WeatherPy/weather_df.csv"
df = pd.read_csv(weather_csv)
clean_df = df.dropna()
del clean_df["Unnamed: 0"]
clean_df.head(20)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Castro,-24.7911,-50.0119,56.53,91,75,7.78,BR,1638609499
1,Necochea,-38.5473,-58.7368,46.04,78,1,8.12,AR,1638609661
2,Ust-Nera,64.5667,143.2000,-37.39,99,69,3.44,RU,1638609497
3,Martapura,-3.4167,114.8500,86.09,74,40,6.91,ID,1638609661
4,Albany,42.6001,-73.9662,31.68,76,65,2.48,US,1638609489
5,Geraldton,-28.7667,114.6000,81.01,50,0,21.85,AU,1638609514
6,Kahului,20.8947,-156.4700,71.83,77,75,16.11,US,1638609506
7,Ushuaia,-54.8000,-68.3000,46.06,53,20,9.22,AR,1638609315
8,Cape Town,-33.9258,18.4232,71.11,52,0,5.99,ZA,1638609482
9,Airai,-8.9266,125.4092,65.07,92,100,0.16,TL,1638609500


In [3]:
gmaps.configure(api_key=g_key)

locations = clean_df[["Lat", "Lng"]]
humidity = clean_df["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Zero cloudiness
no_cloud = clean_df[clean_df["Cloudiness"] == 0]
# Max temp above 70 degrees F
above_70_degrees = no_cloud[no_cloud["Max Temp"] > 70]
# Max temp below 80 degrees F
below_80_degrees = above_70_degrees[above_70_degrees["Max Temp"] < 80]
# Wind speed below 10 mph
below_10_wind = below_80_degrees[below_80_degrees["Wind Speed"] < 10]

# Set Index
filtered_df = below_10_wind.reset_index()
del filtered_df["index"]
filtered_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cape Town,-33.9258,18.4232,71.11,52,0,5.99,ZA,1638609482
1,Zhuhai,22.2769,113.5678,72.86,39,0,3.44,CN,1638609663
2,Faya,18.3851,42.4509,71.71,15,0,1.57,SA,1638609497
3,Boa Vista,2.8197,-60.6733,75.18,83,0,8.05,BR,1638609519
4,Podor,16.6527,-14.9587,73.35,31,0,2.30,MR,1638609667
5,Hervey Bay,-25.2986,152.8535,79.03,70,0,1.01,AU,1638609676
6,Abhā,18.2164,42.5053,71.42,8,0,4.61,SA,1638609677
7,Marzuq,14.4000,46.4667,71.46,21,0,7.18,YE,1638609677
8,Cayenne,4.9333,-52.3333,76.08,89,0,2.30,GF,1638609367
9,Kutum,14.2000,24.6667,77.97,19,0,7.92,SD,1638609512


In [17]:
# Hotel variable
hotels = []

for city in range(len(filtered_df["City"])):

    lat = filtered_df.loc[city]["Lat"]
    lng = filtered_df.loc[city]["Lng"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    request = requests.get(url, params=params)
    response = request.json()

    try:
        hotels.append(response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
filtered_df["Nearest Hotel"] = hotels
filtered_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearest Hotel
0,Cape Town,-33.9258,18.4232,71.11,52,0,5.99,ZA,1638609482,Nearest hotel not found
1,Zhuhai,22.2769,113.5678,72.86,39,0,3.44,CN,1638609663,Nearest hotel not found
2,Faya,18.3851,42.4509,71.71,15,0,1.57,SA,1638609497,Nearest hotel not found
3,Boa Vista,2.8197,-60.6733,75.18,83,0,8.05,BR,1638609519,Nearest hotel not found
4,Podor,16.6527,-14.9587,73.35,31,0,2.30,MR,1638609667,Nearest hotel not found
5,Hervey Bay,-25.2986,152.8535,79.03,70,0,1.01,AU,1638609676,Nearest hotel not found
6,Abhā,18.2164,42.5053,71.42,8,0,4.61,SA,1638609677,Nearest hotel not found
7,Marzuq,14.4000,46.4667,71.46,21,0,7.18,YE,1638609677,Nearest hotel not found
8,Cayenne,4.9333,-52.3333,76.08,89,0,2.30,GF,1638609367,Nearest hotel not found
9,Kutum,14.2000,24.6667,77.97,19,0,7.92,SD,1638609512,Nearest hotel not found


In [19]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in filtered_df.iterrows()]
locations = filtered_df[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))